In [ ]:
import pandas as pd #TO import and manage datasets
import glob
import os
from google.colab import drive
drive.mount('/content/drive')
import re
import os
import csv
import requests
from bs4 import BeautifulSoup as bs
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_folder_path = '/content/drive/MyDrive/dataasetcurrent/'

In [ ]:
output_folder_path = '/content/drive/MyDrive/dissertation/webscrape'

In [ ]:
total_rows = 0
for file_name in os.listdir(input_folder_path):
        file_path = os.path.join(input_folder_path, file_name)
        with open(file_path, "r") as input_file:
          input_df = pd.DataFrame(input_file)
          rows = len(input_df)
          total_rows += rows
          print(f"{file_name}: {rows} rows")

print(f"Total rows in all Excel files: {total_rows}")

Total rows in all Excel files: 0


In [ ]:

def get_Column(comp,val1,val2,urlstatdet):
  label_list=[]
  statdict = { "stats":"standard","keepers":"keeper","keepersadv":"keeper_adv","shooting" : "shooting", "passing": "passing", "pass" : "passing_types", "gca":"gca" , "defense":"defense", "possession": "possession","playingtime":"playing_time" , "misc":"misc"}
  #statdict = {"defense":"defense"}
  if(urlstatdet == "pass"):
    url = f"https://fbref.com/en/comps/{comp}/{val1}-{val2}/passing_types/"
  else:
    url = f"https://fbref.com/en/comps/{comp}/{val1}-{val2}/{urlstatdet}/"
  print(url)
  time.sleep(5)
  res = requests.get(url,timeout=10)
  if res.status_code == 200:
    #print("Response content:")
    #print(res.text)
    pass
  else:
    print(f"Request failed with status code: {res.status_code}")
  #print(res)
  comm = re.compile("<!--|-->")
  soup = bs(comm.sub("", res.text), 'lxml')
  scrapestatdet = statdict[urlstatdet]
  table_det = "stats_" + scrapestatdet
  print(table_det)
  table1 = soup.find("table", id=table_det)
   #table1 = soup.find("table", id='stats_misc')
  #print(table1)
  tr_tag = table1.find("thead")
  tr_tag = tr_tag.find_all("tr")
  tr_tag = tr_tag[1]
  labels = tr_tag.find_all("th")
    #labels = labels.find_all("aria-label")


  for l in labels:
      val = l.get("data-stat")
      print(val)

      label_list.append(val)


  return label_list

In [ ]:
stat_dict_check={}
for filename in os.listdir(input_folder_path):
    if filename.endswith(".csv"):
        input_file_path = os.path.join(input_folder_path, filename)
        output_file_path = os.path.join(output_folder_path, filename)

        # Open the input CSV file
        with open(input_file_path, "r") as input_file:
            csv_reader = csv.reader(input_file)

            # Skip the first row
            next(csv_reader)

            # Get the header row
            file = filename.split(".")[0]
            print("filename is:  ", filename)
            end_year = file.split("-")[1]
            start = file.split("-")[0]
            start_year = start[-4:]
            string = start[:-4]

            number = re.findall(r'\d+', string)[0]

            if "passing_type" in file:
              extracted_text = "pass"
            else:
              pattern = r"([a-zA-Z]+)"

              match_pat = re.search(pattern, string)
              extracted_text = match_pat.group(1)
            for i in stat_dict_check:
              print(i)
            print("extracted text is:  ", extracted_text)
           # print(match_pat)

            if number != 8 and number!=19:
              if extracted_text not in stat_dict_check:
                labels = get_Column(number,start_year,end_year,extracted_text)
                stat_dict_check[extracted_text]=labels
                print(labels)
              else:
                labels = stat_dict_check[extracted_text]


            header_row = labels



            # Find the index of the column containing "Matches" in the header row
            matches_column_index = None
            for i, column_name in enumerate(header_row):
                if "matches" in column_name:
                    matches_column_index = i
                    break

            # Create a list to store the modified rows
            modified_rows = []

            # Process each row and remove the first column and the "Matches" column
            for row in csv_reader:
                modified_row = row[1:matches_column_index] + row[matches_column_index + 1:]
                modified_rows.append(modified_row)

        # Write the modified data to the output CSV file
        with open(output_file_path, "w", newline="") as output_file:
            csv_writer = csv.writer(output_file)

            # Write the modified header row to the output file
            csv_writer.writerow(header_row[1:matches_column_index] + header_row[matches_column_index + 1:])

            # Write the modified rows to the output file
            csv_writer.writerows(modified_rows)



In [ ]:
import csv
import os

# Specify the path to the folder
output_folder_path = '/content/drive/MyDrive/dissertation/webscrape'

original_data_path = '/content/drive/MyDrive/dissertation/datacheck'
#original_data_path = '/content/drive/MyDrive/dissertation/datanew/'

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(output_folder_path) if file.endswith('.csv')]

# Iterate over each CSV file
for csv_file_name in csv_files:
    # Create the full file path
    csv_file_path = os.path.join(output_folder_path, csv_file_name)

    # Create a temporary file to store the modified CSV data
    orig_file_path = os.path.join(original_data_path, csv_file_name)

    # Open the input CSV file for reading
    with open(csv_file_path, 'r', encoding='utf-8') as input_file:

        # Read the CSV data
        csv_reader = csv.reader(input_file)

        # Open the temporary file for writing
        with open(orig_file_path, 'w', newline='' ,encoding='utf-8') as orig_file:

            # Create a CSV writer object
            csv_writer = csv.writer(orig_file)
            #passing_list = ['player','nation','pos','squad','age','born','90s','Passes_attempted','Live_passes','dead_passes','passes_from_free_kicks','through_balls','switches','crosses','throw_ins_taken','corner_kicks','inswing_corner_kicks','outswing_corner_kicks','straight_corner_kicks','passes_completed','passes_offside','passes_blocked']

            # Iterate over each row in the CSV data
            for i, row in enumerate(csv_reader):

                # Check if it is the first row or if the first column contains "Player"
                if i == 0 or row[0] != 'Player':
                    csv_writer.writerow(row)

    #os.remove(csv_file_path)

    # Rename the temporary file to the original file name
    #os.rename(temp_file_path, csv_file_path)

print("Rows with 'Player' in the first column removed for all CSV files in the folder.")

Rows with 'Player' in the first column removed for all CSV files in the folder.


In [ ]:
folder_path = "/content/drive/MyDrive/dissertation/datacheck"
#folder_path = "/content/drive/MyDrive/dissertation/datanew"

csv_files = glob.glob(folder_path + '/*.csv')

In [ ]:

def drop_col(df,file):
    if "stats" in file:
        stat_list = ["xg","npxg","xg_assist","npxg_xg_assist","goals_pens_per90", "xg_per90", "xg_assist_per90", "xg_xg_assist_per90", "npxg_per90","npxg_xg_assist_per90"]
        columns_to_drop = [stat_to_drop for stat_to_drop in stat_list if stat_to_drop in df.columns]
        df = df.drop(columns=columns_to_drop)


    elif "passing" in file:
          pass_list = ['nationality','age','birth_year','minute_90s','assist','xg_assist','pass_xa','xg_assist_net','progressive_passes']
          columns_to_drop = [stat_to_drop for stat_to_drop in pass_list if stat_to_drop in df.columns]
          df = df.drop(columns = columns_to_drop)

    elif "misc" in file:
          pass_type_list = ['nationality','age','birth_year','minute_90s','cards_yellow','cards_red']
          columns_to_drop = [stat_to_drop for stat_to_drop in pass_type_list if stat_to_drop in df.columns]
          df = df.drop(columns = columns_to_drop)

    elif "keepersadv" in file:
          keepers_adv_list = ['nationality','age','birth_year','minute_90s','cards_yellow','cards_red',]
          columns_to_drop = [stat_to_drop for stat_to_drop in keepers_adv_list if stat_to_drop in df.columns]
          df = df.drop(columns = columns_to_drop)
    elif "keepers" in file:
          keepers_list =['nationality','age','birth_year','minute_90s','gk_goals_against']
          columns_to_drop = [stat_to_drop for stat_to_drop in keepers_list if stat_to_drop in df.columns]
          df.drop(columns=columns_to_drop)

    elif "gca" in file:
          gca_list = ['nationality','age','birth_year','minutes_90s']
          columns_to_drop = [stat_to_drop for stat_to_drop in gca_list if stat_to_drop in df.columns]
          df.drop(columns = columns_to_drop)

    elif "defense" in file:

          defense_list = ['nationality','age','birth_year','minute_90s','takles_won','interceptions']
          columns_to_drop = [stat_to_drop for stat_to_drop in defense_list if stat_to_drop in df.columns]
          df.drop(columns = columns_to_drop)
    elif "shooting" in file:
        shooting_list = ["player","position","team","shots","shots_on_target","average_shot_distance","shots_per90","goals_per_shot_on_target","goals_per_shot","shots_on_target_per90","shots_free_kicks"]
        cols = df.columns
        for col in cols:
          if col not in shooting_list:
            df = df.drop(columns=col)
        #print(df)

    elif "possession" in file:
        possession_list = ['player','position','team','touches','touches_def_pen_area', 'touches_def_3rd', 'touches_mid_3rd','touches_att_3rd', 'touches_att_pen_area', 'touches_live_ball', 'take_ons', ' take_ons_won','take_ons_tackled','carries','carries_distance','carries_progressive_distance','carries_into_final_third','carries_into_penalty_area','miscontrols','dispossessed','passes_received' ]
        cols = df.columns
        for col in cols:
          if col not in possession_list:
            df = df.drop(columns=col)

    elif "playingtime" in file:
        pt_list = ['player','position','team','minutes_per_game','minutes_per_start','games_complete','games_subs','minutes_per_sub','unused_subs','points_per_game','on_goals_for','on_goals_against']
        cols = df.columns
        for col in cols:
          if col not in pt_list:
            df = df.drop(columns=col)

    elif "passing_types" in file:
        passing_type_list = ['nationality','age','birth_year','minute_90s','crosses','passes_completed']
        columns_to_drop = [stat_to_drop for stat_to_drop in passing_type_list if stat_to_drop in df.columns]
        df = df.drop(columns = columns_to_drop)

    return df



In [ ]:
groupedDF = pd.DataFrame()

In [ ]:
groupedlist_year1 =[]
groupedlist_year2 =[]
groupedlist_year3 =[]
groupedlist_year4 =[]
groupedlist_year4 =[]
groupedlist_year5 =[]
def grouped_func(df,file):
  if "2017-2018" in file:
    print("file in 2017",file)
    groupedlist_year1.append(df)
  elif "2018-2019" in file:
    print("file in 2018",file)
    groupedlist_year2.append(df)
  elif "2019-2020" in file:
    print("file in 2019",file)
    groupedlist_year3.append(df)
  elif "2020-2021" in file:
    print("file in 2020",file)
    groupedlist_year4.append(df)
  else:
    print("file in 2021",file)
    groupedlist_year5.append(df)


In [ ]:
path = r'/content/drive/MyDrive/dissertation/datacheck'
#path = r'/content/drive/MyDrive/dissertation/datanew'

folder = glob.glob(path + "/*.csv")

In [ ]:
i=0
#appending the excel to yearwise list based on the excel's name
for file in folder:
  pd.set_option('display.max_columns', None)
  specificdf = pd.read_csv(file,encoding='utf-8')
  league = os.path.basename(file)
  #print("l",league)
  filename = league.split('.')[0] # filename
  #print(filename)
  filer = filename.split(".")[0]
  end_year = filer.split("-")[1] #year 2
  start = filer.split("-")[0] #eg:defense82019
  start_year = start[-4:] #year1
  string = start[:-4] #category and comp id
  number = re.findall(r'\d+', string)[0]    #compid
  number = int(number)
  #print("number is",number)
  pattern = r"([a-zA-Z]+)"
  match_pat = re.search(pattern, string)
  extracted_text = match_pat.group(1)  #only category
  year = start_year + "-" + end_year
  specificdf['year'] = year
  specificdf['competition'] = number
  specificdf = drop_col(specificdf,filename)
  #print(specificdf)
  #print("start is", start)
  if number==8 or number==19:
    continue

  grouped_func(specificdf,filer)


file in 2021 stats92021-2022
file in 2021 shooting92021-2022
file in 2021 keepers92021-2022
file in 2021 defense92021-2022
file in 2021 passing_types92021-2022
file in 2021 gca92021-2022
file in 2021 possession92021-2022
file in 2021 playingtime92021-2022
file in 2021 keepersadv92021-2022
file in 2021 passing92021-2022
file in 2021 misc92021-2022
file in 2021 keepers122021-2022
file in 2021 passing122021-2022
file in 2021 possession122021-2022
file in 2021 playingtime122021-2022
file in 2021 passing_types122021-2022
file in 2021 stats122021-2022
file in 2021 keepersadv122021-2022
file in 2021 misc122021-2022
file in 2021 shooting122021-2022
file in 2021 defense122017-2021
file in 2021 gca122021-2022
file in 2017 defense202017-2018
file in 2017 defense122017-2018
file in 2018 defense112018-2019
file in 2017 defense132017-2018
file in 2018 defense132018-2019
file in 2017 defense112017-2018
file in 2018 defense122018-2019
file in 2019 defense202019-2020
file in 2019 defense112019-2020
fil

In [ ]:
#filepath = ["/content/drive/MyDrive/dissertation/year1.csv","/content/drive/MyDrive/dissertation/year2.csv","/content/drive/MyDrive/dissertation/year3.csv","/content/drive/MyDrive/dissertation/year4.csv","/content/drive/MyDrive/dissertation/year5.csv"]

def to_csv(df):

      df.to_csv("/content/drive/MyDrive/dissertation/year1.csv")

In [ ]:
year1DF = pd.DataFrame()
year1DF = pd.concat(groupedlist_year1)
year1DF = year1DF.groupby('player')
year1DF.apply(to_csv)
year2DF = pd.DataFrame()
year2DF = pd.concat(groupedlist_year2)
year2DF = year2DF.groupby('player')
year2DF.apply(to_csv)
year3DF = pd.DataFrame()
year3DF = pd.concat(groupedlist_year3)
year3DF = year3DF.groupby('player')
year3DF.apply(to_csv)
year4DF = pd.DataFrame()
year4DF = pd.concat(groupedlist_year4)
year4DF = year4DF.groupby('player')
year4DF.apply(to_csv)
year5DF = pd.DataFrame()
year5DF = pd.concat(groupedlist_year5)
year5DF = year5DF.groupby('player')
year5DF.apply(to_csv)



""


In [ ]:
year1DF = pd.DataFrame()
year1DF = pd.concat(groupedlist_year1,ignore_index=True)

year1DF.to_csv("/content/drive/MyDrive/dissertation/year1check.csv")

year2DF = pd.DataFrame()
year2DF = pd.concat(groupedlist_year2,ignore_index=True)
year2DF.to_csv("/content/drive/MyDrive/dissertation/year2check.csv")

year3DF = pd.DataFrame()
year3DF = pd.concat(groupedlist_year3,ignore_index=True)
year3DF.to_csv("/content/drive/MyDrive/dissertation/year3check.csv")

year4DF = pd.DataFrame()
year4DF = pd.concat(groupedlist_year4,ignore_index=True)
year4DF.to_csv("/content/drive/MyDrive/dissertation/year4check.csv")

year5DF = pd.DataFrame()
year5DF = pd.concat(groupedlist_year5,ignore_index=True)
year5DF.to_csv("/content/drive/MyDrive/dissertation/year5check.csv")

In [ ]:
pivot_table_columns = year1DF.columns.tolist()
pivot_table_columns.remove("player")
year1pivot = year1DF.pivot_table(index=['player'],
                     values = pivot_table_columns, aggfunc='first').reset_index()
year2pivot = year2DF.pivot_table(index=['player'],
                     values = pivot_table_columns, aggfunc='first').reset_index()
year3pivot = year3DF.pivot_table(index=['player'],
                     values = pivot_table_columns, aggfunc='first').reset_index()
year4pivot = year4DF.pivot_table(index=['player'],
                     values = pivot_table_columns, aggfunc='first').reset_index()
year5pivot = year5DF.pivot_table(index=['player'],
                     values = pivot_table_columns, aggfunc='first').reset_index()
#print(type(year1pivot))
year1pivot.to_csv("/content/drive/MyDrive/dissertation/year1write.csv")
year2pivot.to_csv("/content/drive/MyDrive/dissertation/year2write.csv")
year3pivot.to_csv("/content/drive/MyDrive/dissertation/year3write.csv")
year4pivot.to_csv("/content/drive/MyDrive/dissertation/year4write.csv")
year5pivot.to_csv("/content/drive/MyDrive/dissertation/year5write.csv")

In [ ]:
grouped_player_df = pd.concat([year1DF,year2DF,year3DF,year4DF,year5DF])

In [ ]:
grouped_player_df.to_csv("/content/drive/MyDrive/dissertation/grouped_player_df3.csv")

In [ ]:
print(len(grouped_player_df))

15773


In [ ]:
groupedDF = groupedDF.replace("NaN", 0)